In this section, we  consider the famous Gauss-Markov problem which will give
us an opportunity to use all the material we have so far developed. The
Gauss-Markov model is the fundamental model for noisy parameter estimation because it
estimates unobservable parameters given a noisy indirect measurement.
Incarnations of the same model appear in all studies of Gaussian models. This
case is an excellent opportunity to use everything we have so far learned about
projection and conditional expectation.

Following Luenberger [[luenberger1968optimization]](#luenberger1968optimization) let's  consider the
following problem:

$$
\mathbf{y} = \mathbf{W} \boldsymbol{\upbeta} + \boldsymbol{\epsilon}
$$

 where $\mathbf{W}$ is a $ n \times m $ matrix, and $\mathbf{y}$ is a
$n \times 1$ vector. Also, $\boldsymbol{\epsilon}$ is a $n$-dimensional normally
distributed random vector with zero-mean and covariance,

$$
\mathbb{E}( \boldsymbol{\epsilon} \boldsymbol{\epsilon}^T) = \mathbf{Q}
$$

 Note that engineering systems usually provide a *calibration mode*
where you can estimate $\mathbf{Q}$ so it's not fantastical to assume you have
some knowledge of the noise statistics. The problem is to find a matrix
$\mathbf{K}$ so that $ \boldsymbol{\hat{\upbeta}}=\mathbf{K}^T \mathbf{y}$
approximates $ \boldsymbol{\upbeta}$.  Note that we only have knowledge of
$\boldsymbol{\upbeta}$ via $ \mathbf{y}$ so we can't measure it directly.
Further, note that $\mathbf{K} $ is a matrix, not a vector, so there are $m
\times n$ entries to compute. 

We can approach this problem the usual way by trying to solve the MMSE
problem:

$$
\min_K\mathbb{E}(\Vert\boldsymbol{\hat{\upbeta}}-\boldsymbol{\upbeta} \Vert^2)
$$

 which we can write out as

$$
\min_K\mathbb{E}(\Vert\boldsymbol{\hat{\upbeta}}-\boldsymbol{\upbeta} \Vert^2) =  \min_K\mathbb{E}(\Vert \mathbf{K}^T\mathbf{y}- \boldsymbol{\upbeta} \Vert^2) =  \min_K\mathbb{E}(\Vert \mathbf{K}^T\mathbf{W}\mathbf{\boldsymbol{\upbeta}}+\mathbf{K}^T\boldsymbol{\epsilon}- \boldsymbol{\upbeta} \Vert^2)
$$

 and since $\boldsymbol{\epsilon}$ is the only random variable here,
this simplifies to

$$
\min_K \Vert \mathbf{K}^T\mathbf{W}\mathbf{\boldsymbol{\upbeta}}-\boldsymbol{\upbeta} \Vert^2 + \mathbb{E}(\Vert\mathbf{K}^T\boldsymbol{\epsilon} \Vert^2)
$$

 The next step is to compute

$$
\mathbb{E}(\Vert\mathbf{K}^T\boldsymbol{\epsilon} \Vert^2) =\Tr \mathbb{E}(\mathbf{K}^T\boldsymbol{\epsilon}\boldsymbol{\epsilon}^T\mathbf{K})=\Tr(\mathbf{K^T Q K})
$$

 using the properties of the trace of  a matrix. We can assemble
everything as

$$
\min_K\Vert\mathbf{K^T W}\boldsymbol{\upbeta}-\boldsymbol{\upbeta}\Vert^2+\Tr(\mathbf{K^T Q K})
$$

 Now, if we were to solve this for $\mathbf{K}$, it would be a
function of $ \boldsymbol{\upbeta}$, which is the same thing as saying that the
estimator, $ \boldsymbol{\hat{\upbeta}}$, is a function of what we are trying to
estimate, $\boldsymbol{\upbeta}$, which makes no sense. However, writing this out
tells us that if we had $\mathbf{K^T W}= \mathbf{I}$, then the first term
vanishes and the problem simplifies to

$$
\min_K \Tr(\mathbf{K^T Q K})
$$

 with the contraint,

$$
\mathbf{K^T W} = \mathbf{I}
$$

 This requirement is the same as asserting that the estimator is unbiased,

$$
\mathbb{E}(\boldsymbol{\hat{\upbeta}})=\mathbf{K^T W}  \boldsymbol{\upbeta} =  \boldsymbol{\upbeta}
$$

 To line this problem up with our earlier work, let's consider  the
$i^{th}$ column of $\mathbf{K}$, $\mathbf{k}_i$. Now, we can re-write the
problem as

$$
\min_k (\mathbf{k}_i^T \mathbf{Q} \mathbf{k}_i)
$$

 with

$$
\mathbf{W}^T \mathbf{k}_i = \mathbf{e}_i
$$

 and we know how to solve this from our previous work on contrained optimization,

$$
\mathbf{k}_i = \mathbf{Q}^{-1}\mathbf{W} (\mathbf{W^T Q^{-1} W})^{-1}  \mathbf{e}_i
$$

 Now all we have to do is stack these together for the general solution:

$$
\mathbf{K} =  \mathbf{Q}^{-1}\mathbf{W} (\mathbf{W^T Q^{-1} W})^{-1}
$$

 It's easy when you have all of the concepts lined up! For completeness, the
covariance of the error is

$$
\mathbb{E}(\hat{\boldsymbol{\upbeta}}-\boldsymbol{\upbeta}) (\hat{\boldsymbol{\upbeta}}-\boldsymbol{\upbeta})^T =\mathbf{K}^T\mathbf{Q}\mathbf{K} =(\mathbf{W}^T \mathbf{Q}^{-1} \mathbf{W})^{-1}
$$

<!-- # !bc pycod -->
<!-- # from mpl_toolkits.mplot3d import proj3d -->
<!-- # from numpy.linalg import inv -->
<!-- # import matplotlib.pyplot as plt -->
<!-- # import numpy as np -->
<!-- # from numpy import matrix, linalg, ones, array -->
<!-- # Q = np.eye(3)*.1 # error covariance matrix -->
<!-- # beta = matrix(ones((2,1))) # this is what we are trying estimate -->
<!-- # W = matrix([[1,2], -->
<!-- #             [2,3], -->
<!-- #             [1,1]]) -->
<!-- # ntrials = 50 -->
<!-- # epsilon = np.random.multivariate_normal((0,0,0),Q,ntrials).T -->
<!-- # y=W*beta+epsilon -->
<!-- # -->
<!-- # K=inv(W.T*inv(Q)*W)*matrix(W.T)*inv(Q) -->
<!-- # b=K*y #estimated beta from data -->
<!-- # -->
<!-- # fig = plt.figure() -->
<!-- # fig.set_size_inches([6,6]) -->
<!-- # -->
<!-- # # some convenience definitions for plotting -->
<!-- # bb = array(b) -->
<!-- # bm = bb.mean(1) -->
<!-- # yy = array(y) -->
<!-- # ax = fig.add_subplot(111, projection='3d') -->
<!-- # -->
<!-- # ax.plot3D(yy[0,:],yy[1,:],yy[2,:],'mo',label='y',alpha=0.3) -->
<!-- # ax.plot3D([beta[0,0],0],[beta[1,0],0],[0,0],'r-',label=r'$\upbeta$') -->
<!-- # ax.plot3D([bm[0],0],[bm[1],0],[0,0],'g-',lw=1,label=r'$\widehat{\upbeta}_m$') -->
<!-- # ax.plot3D(bb[0,:],bb[1,:],0*bb[1,:],'.g',alpha=0.5,lw=3,label=r'$\hat{\upbeta}$') -->
<!-- # ax.legend(loc=0,fontsize=18) -->
<!-- # plt.show() -->
<!-- # !ec -->

<!-- dom:FIGURE: [fig-statistics/Gauss_Markov_001.png, width=500 frac=0.85] The red circles show the points to be estimated in the *xy*-plane by the black points. <div id="fig:Gauss_Markov_001"></div> -->
<!-- begin figure -->
<div id="fig:Gauss_Markov_001"></div>

<p>The red circles show the points to be estimated in the <em>xy</em>-plane by the black points.</p>
<img src="fig-statistics/Gauss_Markov_001.png" width=500>

<!-- end figure -->


[Figure](#fig:Gauss_Markov_001) shows the simulated $\mathbf{y}$ data as red
circles. The black dots show the corresponding estimates,
$\boldsymbol{\hat{\upbeta}}$ for each sample. The black lines show the true value
of $\boldsymbol{\upbeta}$ versus the average of the estimated
$\boldsymbol{\upbeta}$-values, $\widehat{\boldsymbol{\upbeta}_m}$. The matrix
$\mathbf{K}$ maps the red circles in the corresponding dots. Note there are
many possible ways to map the red circles to the plane, but the $\mathbf{K}$ is
the one that minimizes the MSE for $\boldsymbol{\upbeta}$. 

**Programming Tip.**

The following snippets provide a quick code walkthrough.
To simulate the target data, we define the relevant 
matrices below,

In [1]:
Q = np.eye(3)*0.1 # error covariance matrix
# this is what we are trying estimate
beta = matrix(ones((2,1))) 
W = matrix([[1,2],
            [2,3],
            [1,1]])

  Then, we generate the noise terms and create
the simulated data, $y$,

In [2]:
ntrials = 50 
epsilon = np.random.multivariate_normal((0,0,0),Q,ntrials).T 
y=W*beta+epsilon

<!-- dom:FIGURE: [fig-statistics/Gauss_Markov_002.png, width=500 frac=0.85] Focusing on the *xy*-plane in [Figure](#fig:Gauss_Markov_001), the dashed line shows the true value for $\boldsymbol{\upbeta}$ versus the mean of the estimated values $\widehat{\boldsymbol{\upbeta}}_m$. <div id="fig:Gauss_Markov_002"></div> -->
<!-- begin figure -->
<div id="fig:Gauss_Markov_002"></div>

<p>Focusing on the <em>xy</em>-plane in [Figure](#fig:Gauss_Markov_001), the dashed line shows the true value for $\boldsymbol{\upbeta}$ versus the mean of the estimated values $\widehat{\boldsymbol{\upbeta}}_m$.</p>
<img src="fig-statistics/Gauss_Markov_002.png" width=500>

<!-- end figure -->


[Figure](#fig:Gauss_Markov_002) shows more detail in the horizontal *xy*-plane
of [Figure](#fig:Gauss_Markov_001).  [Figure](#fig:Gauss_Markov_002) shows
the dots, which are individual estimates of $\boldsymbol{\hat{\upbeta}}$ from the
corresponding simulated $\mathbf{y}$ data. The dashed line is the true value
for $\boldsymbol{\upbeta}$ and the filled line ($\widehat{\boldsymbol{\upbeta}}_m$)
is the average of all the dots.  The gray ellipse provides an error ellipse
for the covariance of the estimated $\boldsymbol{\upbeta}$ values.  

**Programming Tip.**

The following snippets provide a quick walkthrough of
the construction of [Figure](#fig:Gauss_Markov_002). To draw
the ellipse, we need to import the patch primitive,

In [3]:
%matplotlib inline

from  matplotlib.patches import Ellipse

 To compute the parameters of the error ellipse based on the
covariance matrix of the individual estimates of $\boldsymbol{\upbeta}$
in the `bm_cov` variable below,

In [4]:
U,S,V = linalg.svd(bm_cov) 
err = np.sqrt((matrix(bm))*(bm_cov)*(matrix(bm).T))
theta = np.arccos(U[0,1])/np.pi*180

 Then, we draw the add the scaled ellipse
in the following,

In [5]:
ax.add_patch(Ellipse(bm,err*2/np.sqrt(S[0]),
                     err*2/np.sqrt(S[1]),
                     angle=theta,color='gray'))

<!-- References -->
<!-- --------------- -->
<!--  -->
<!-- * Luenberger, David G. *Optimization by vector space methods*. Wiley-Interscience, 1997. -->